In [25]:
#Channel overlaps are only calculated in one direction. 
#If a single channel and all of their connections are required, that can be obtained using this notebook. 

import boto3
import pickle as pkl
import time

overlap_bucket = 'youtube-overlaps'
commenter_bucket = 'youtube-commenters'

commenter_file_name = ""
overlap_file_name = ""

new_file_name = ""


In [26]:
def see_all_files_s3(bucket):
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(bucket)
    files = []
    for obj in s3_bucket.objects.all():
        files.append(obj.key)
    return files

def load_pkl_obj_s3(file_name, bucket):
    try:
        s3 = boto3.resource('s3')
        obj = pkl.loads(s3.Bucket(bucket).Object(file_name).get()['Body'].read())
    except:
        time.sleep(5)
        return load_pkl_obj_s3(file_name, bucket)
        
    return obj

def dump_pkl_obj_s3(obj, file_name, bucket):
    s3 = boto3.resource('s3')
    pickle_obj = pkl.dumps(obj)
    response = s3.Object(bucket, file_name).put(Body=pickle_obj)
    return response['ResponseMetadata']

In [27]:
def calculate_new_overlaps_for_channel(channel_dict, channel_old_overlaps, bucket):
    primary_channel = list(channel_dict.keys())[0]
    primary_channel_commenter_set = set(channel_dict[primary_channel])
    
    utility_files = ['ChannelIdMap.pkl', 'CurrentChannel.pkl', 'YoutubeUsernames.pkl']
    commenter_files = [file for file in see_all_files_s3(bucket) if file not in utility_files]
    channel_file_dict = {}
    for file_name in commenter_files:
        prefix_len = 9
        suffix_len = 17
        channel_name = file_name[prefix_len:-suffix_len]
        if(channel_name not in channel_old_overlaps[primary_channel]):
            channel_file_dict[channel_name] = file_name

    len(channel_file_dict)

    for comparison_channel, comparison_file in channel_file_dict.items():
        print(comparison_channel)

        comparison_channel_commenter_dict = load_pkl_obj_s3(comparison_file, bucket)

        shared_commenters = primary_channel_commenter_set & comparison_channel_commenter_dict[comparison_channel]
        shared_commenter_count = len(shared_commenters)
        channel_old_overlaps[primary_channel][comparison_channel] = shared_commenter_count
    
    
    return channel_old_overlaps


In [28]:
channel_dict = load_pkl_obj_s3(commenter_file_name, commenter_bucket)
channel_old_overlaps = load_pkl_obj_s3(overlap_file_name, overlap_bucket)

new_overlap_dict = calculate_new_overlaps_for_channel(channel_dict, channel_old_overlaps, commenter_bucket)

In [29]:
success = dump_pkl_obj_s3(new_overlap_dict, new_file_name, overlap_bucket)

{'RequestId': '5VZ63F0J1NVYC3CE',
 'HostId': 'Y0Z8YIQQykxek/t1/+iE1vxSgsODehUmAn6ObJesuD4jUmvQcE2BCwy8Z6fVC87cME8Edmn3eh8=',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'x-amz-id-2': 'Y0Z8YIQQykxek/t1/+iE1vxSgsODehUmAn6ObJesuD4jUmvQcE2BCwy8Z6fVC87cME8Edmn3eh8=',
  'x-amz-request-id': '5VZ63F0J1NVYC3CE',
  'date': 'Tue, 14 Dec 2021 22:34:32 GMT',
  'etag': '"0fdd80889d17bed6756fb2fff18e740d"',
  'server': 'AmazonS3',
  'content-length': '0'},
 'RetryAttempts': 1}

In [30]:
#Verification
channel_dict = load_pkl_obj_s3(new_file_name, overlap_bucket)
print(len(channel_dict[list(channel_dict.keys())[0]]))

5696
